# Avaliação das notas de matemática do ENEM 2016

### Nesse exercício serão utilizados dados públicos disponibilizados pela INEP para construir um módelo que preeveja quanto um aluno obteria na prova de matemática

## Importando as bibliotecas necessárias

In [3]:
import numpy as np
import pandas as pd
import re
from tpot import TPOTRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor

## Importando o dataset

In [4]:
# Importando o dataset
df = pd.read_csv('train.csv', index_col=0)
X_test_df = pd.read_csv('test.csv')

## Analise exploratório dos dados

In [5]:
df.head()

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,...,Q041,Q042,Q043,Q044,Q045,Q046,Q047,Q048,Q049,Q050
1,ed50e8aaa58e7a806c337585efee9ca41f1eb1ad,2016,4314902,Porto Alegre,43,RS,24,M,0.0,1,...,5.0,A,A,A,A,A,A,A,B,D
2,2c3acac4b33ec2b195d77e7c04a2d75727fad723,2016,2304707,Granja,23,CE,17,F,0.0,3,...,NaN,A,A,C,A,B,A,A,C,A
3,f4545f8ccb9ff5c8aad7d32951b3f251a26e6568,2016,2304400,Fortaleza,23,CE,21,F,0.0,3,...,NaN,A,A,A,A,C,A,A,B,A
4,3d6ec248fef899c414e77f82d5c6d2bffbeaf7fe,2016,3304557,Rio de Janeiro,33,RJ,25,F,0.0,0,...,5.0,C,A,A,A,A,D,A,A,A
5,bf896ac8d3ecadd6dba1dfbf50110afcbf5d3268,2016,1302603,Manaus,13,AM,28,M,0.0,2,...,NaN,A,A,A,A,A,A,A,A,A


In [6]:
X_test_df.shape

(4576, 47)

In [7]:
df_aux = pd.DataFrame({'Colunas': df.columns, 'Tipos': df.dtypes, 'Na_count': df.isna().sum(), 
                       'Na_percent': df.isna().sum()/df.shape[0]})
df_aux.sort_values('Na_percent', ascending=False)[:10]

,Colunas,Tipos,Na_count,Na_percent
SG_UF_ENTIDADE_CERTIFICACAO,SG_UF_ENTIDADE_CERTIFICACAO,object,12092,0.880699
CO_UF_ENTIDADE_CERTIFICACAO,CO_UF_ENTIDADE_CERTIFICACAO,float64,12092,0.880699
NO_ENTIDADE_CERTIFICACAO,NO_ENTIDADE_CERTIFICACAO,object,12092,0.880699
Q041,Q041,float64,10792,0.786016
TP_SIT_FUNC_ESC,TP_SIT_FUNC_ESC,float64,9448,0.688128
TP_LOCALIZACAO_ESC,TP_LOCALIZACAO_ESC,float64,9448,0.688128
TP_DEPENDENCIA_ADM_ESC,TP_DEPENDENCIA_ADM_ESC,float64,9448,0.688128
SG_UF_ESC,SG_UF_ESC,object,9448,0.688128
CO_UF_ESC,CO_UF_ESC,float64,9448,0.688128
NO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,object,9448,0.688128


### Começando o pre-processamento dos dados

In [8]:
# Substituindo as notas ausentes da coluna 'NU_NOTA_MT' para 0
df['NU_NOTA_MT'].replace(np.nan, 0, inplace=True)

In [9]:
# Algumas colunas tem mais de 50% de dados faltantes, imputar os dados não traria beneficio ao modelo de ML, 
# assim essas colunas serão deletadas

colComMuitosDadosFaltantes = [*df_aux [df_aux['Na_percent'] >= 0.5]['Colunas'].values]
print('Deletando {} colunas do dataset de treino e teste, porque mais de 50% de dados estão ausentes'.format(len(colComMuitosDadosFaltantes)))
print()
df.drop(colComMuitosDadosFaltantes, axis=1, inplace=True)
print('Colunas deletadas: {}'.format(colComMuitosDadosFaltantes))

col = [i for i in X_test_df.columns if i in colComMuitosDadosFaltantes]
X_test_df.drop(col, axis=1, inplace=True)

Deletando 20 colunas do dataset de treino e teste, porque mais de 50% de dados estão ausentes

Colunas deletadas: ['TP_ENSINO', 'CO_ESCOLA', 'CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC', 'CO_UF_ESC', 'SG_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC', 'NO_ENTIDADE_CERTIFICACAO', 'CO_UF_ENTIDADE_CERTIFICACAO', 'SG_UF_ENTIDADE_CERTIFICACAO', 'Q027', 'Q028', 'Q029', 'Q030', 'Q031', 'Q032', 'Q033', 'Q041']


In [10]:
# Selecionando apenas as colunas presentes em ambos os datasets + NU_NOTA_MT
df = df[[*X_test_df.columns] + ['NU_NOTA_MT']]

In [11]:
colunas_dropadas = ['NU_INSCRICAO', 'CO_UF_RESIDENCIA', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT']

num = re.compile('NU_')
colunas_numericas = [i for i in filter(num.match, X_test_df.columns) if i not in colunas_dropadas]
print(colunas_numericas)

print()

cat = re.compile('SG_|TP_|IN_|Q0')
colunas_categoricas = [i for i in filter(cat.match, X_test_df.columns) if i not in colunas_dropadas]
print(colunas_categoricas)

['NU_IDADE', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'NU_NOTA_REDACAO']

['SG_UF_RESIDENCIA', 'TP_SEXO', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'IN_TREINEIRO', 'IN_BAIXA_VISAO', 'IN_CEGUEIRA', 'IN_SURDEZ', 'IN_DISLEXIA', 'IN_DISCALCULIA', 'IN_SABATISTA', 'IN_GESTANTE', 'IN_IDOSO', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_LINGUA', 'TP_STATUS_REDACAO', 'Q001', 'Q002', 'Q006', 'Q024', 'Q025', 'Q026', 'Q047']


In [12]:
df.drop(colunas_dropadas, axis=1, inplace=True)

In [14]:
df[colunas_categoricas] = df[colunas_categoricas].astype('category')
X_test_df[colunas_categoricas] = X_test_df[colunas_categoricas].astype('category')

In [15]:
print('O dataset possui {} observações'.format(df.shape[0]))
df2 = df.dropna()
print('Após excluir as linhas com valores inexistentes, o dataset ainda possui {} observações'.format(df2.shape[0]))

O dataset possui 13730 observações
Após excluir as linhas com valores inexistentes, o dataset ainda possui 10097 observações


In [16]:
# Versão sem imputação de dados

preProcessor = ColumnTransformer(transformers=[('cat', OneHotEncoder(), colunas_categoricas), ('num', StandardScaler(), colunas_numericas)])
reg = Pipeline(steps=[('preprocessor', preProcessor), ('regression', LinearRegression())])

In [17]:
# Versão imputando os dados

numeric_transform = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())])
categorical_transform = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')), ('encoder', OneHotEncoder(sparse=False, drop='first'))])
preProcessor2 = ColumnTransformer(transformers=[('cat', categorical_transform, colunas_categoricas), ('num', numeric_transform, colunas_numericas)])
reg2 = Pipeline(steps=[('preprocessor', preProcessor2), ('regression', LinearRegression())])

In [18]:
y_train = df['NU_NOTA_MT']
X_train = df.drop('NU_NOTA_MT', axis=1)
y_train_sem_imp = df2['NU_NOTA_MT']
X_train_sem_imp = df2.drop('NU_NOTA_MT', axis=1)
inscricao = X_test_df['NU_INSCRICAO']
X_test = X_test_df.drop(colunas_dropadas, axis=1)

In [19]:
pd.DataFrame({'column': X_train.columns, 'type': X_train.dtypes, 'na_count': X_train.isna().sum()})

,column,type,na_count
SG_UF_RESIDENCIA,SG_UF_RESIDENCIA,category,0
NU_IDADE,NU_IDADE,int64,0
TP_SEXO,TP_SEXO,category,0
TP_COR_RACA,TP_COR_RACA,category,0
TP_NACIONALIDADE,TP_NACIONALIDADE,category,0
TP_ST_CONCLUSAO,TP_ST_CONCLUSAO,category,0
TP_ANO_CONCLUIU,TP_ANO_CONCLUIU,category,0
TP_ESCOLA,TP_ESCOLA,category,0
IN_TREINEIRO,IN_TREINEIRO,category,0
IN_BAIXA_VISAO,IN_BAIXA_VISAO,category,0


In [20]:
X_test_df.dropna(inplace=True)

In [51]:
# Calculando o erro do modelo com os dados imputados
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_train, y_train, test_size=0.3, random_state=4)

_ = reg2.fit(X_train1, y_train1)

mean_squared_error(y_test1, reg2.predict(X_test1))

4357.543292355168

In [52]:
# Calculando o erro do modelo com os dados não imputados
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_train_sem_imp, y_train_sem_imp, test_size=0.3, random_state=40)

_= reg.fit(X_train1, y_train1)

mean_squared_error(y_test1, reg.predict(X_test1))

5453.369267285228

### Modelo sem imputar os dados - Não utilizado

In [54]:
#_ = reg.fit(X_train_sem_imp, y_train_sem_imp)
#a = reg.predict(X_train.dropna())

#answer = pd.DataFrame({'NU_INSCRICAO': inscricao, 'NU_NOTA_MT': a})
#answer.to_csv('answer.csv', index=False)

In [579]:
#answer = pd.DataFrame({'NU_INSCRICAO': inscricao, 'NU_NOTA_MT': a})
#answer.to_csv('answer1.csv', index=False)

### Modelo imputando os dados

In [55]:
# Modelo imputando os dados
_ = reg2.fit(X_train, y_train)
notas = reg2.predict(X_test)

answer = pd.DataFrame({'NU_INSCRICAO': inscricao, 'NU_NOTA_MT': notas})
answer['NU_NOTA_MT'] = answer['NU_NOTA_MT'].astype('float')

def notaNan(x):
    if x['NU_NOTA_MT'] <= 300:
        return np.nan
    else:
        return round(x['NU_NOTA_MT'],1)

answer['NU_NOTA_MT'] = answer.apply(notaNan, axis=1)
answer.head()

,NU_INSCRICAO,NU_NOTA_MT
0,73ff9fcc02f0a99919906c942c2e1a1042cdcf98,431.5
1,71a95f9f1b91a82c65ad94abbdf9f54e6066f968,442.4
2,b38a03232f43b11c9d0788abaf060f7366053b6d,574.7
3,70b682d9a3636be23f6120fa9d6b164eb3c6002d,NaN
4,715494628a50142ce8cb17191cfe6d0f3cae0934,554.5


In [56]:
answer.to_csv('answer1.csv', index=False)

### Usando TPOTRegressor

In [57]:
# Primeiramente preprocessando os dados de treino
X_train_preProcessed = preProcessor2.fit_transform(X_train)

In [58]:
tpot = TPOTRegressor(generations=5, max_eval_time_mins= 1, population_size=10, verbosity=2)

In [59]:
tpot.fit(X_train_preProcessed, y_train)


Generation 1 - Current best internal CV score: -3982.774932187784
Generation 2 - Current best internal CV score: -3949.093037730353
Generation 3 - Current best internal CV score: -3949.093037730353
Generation 4 - Current best internal CV score: -3949.093037730353
Generation 5 - Current best internal CV score: -3949.093037730353
Best pipeline: RandomForestRegressor(LinearSVR(input_matrix, C=10.0, dual=True, epsilon=0.0001, loss=squared_epsilon_insensitive, tol=0.1), bootstrap=False, max_features=0.2, min_samples_leaf=4, min_samples_split=11, n_estimators=100)


TPOTRegressor(config_dict=None, crossover_rate=0.1, cv=5,
              disable_update_check=False, early_stop=None, generations=5,
              log_file=<ipykernel.iostream.OutStream object at 0x0000012A9580D288>,
              max_eval_time_mins=1, max_time_mins=None, memory=None,
              mutation_rate=0.9, n_jobs=1, offspring_size=None,
              periodic_checkpoint_folder=None, population_size=10,
              random_state=None, scoring=None, subsample=1.0, template=None,
              use_dask=False, verbosity=2, warm_start=False)

In [60]:
X_test_preProcessed = preProcessor2.transform(X_test)

In [61]:
notas2 = tpot.predict(X_test_preProcessed)

In [62]:
answer2 = pd.DataFrame({'NU_INSCRICAO': inscricao, 'NU_NOTA_MT': notas2})
answer2['NU_NOTA_MT'] = answer2['NU_NOTA_MT'].astype('float')

def notaNan(x):
    if x['NU_NOTA_MT'] <= 300:
        return np.nan
    else:
        return round(x['NU_NOTA_MT'], 1)

answer2['NU_NOTA_MT'] = answer2.apply(notaNan, axis=1)
answer2.head()

,NU_INSCRICAO,NU_NOTA_MT
0,73ff9fcc02f0a99919906c942c2e1a1042cdcf98,422.0
1,71a95f9f1b91a82c65ad94abbdf9f54e6066f968,451.5
2,b38a03232f43b11c9d0788abaf060f7366053b6d,584.4
3,70b682d9a3636be23f6120fa9d6b164eb3c6002d,NaN
4,715494628a50142ce8cb17191cfe6d0f3cae0934,560.0


In [63]:
answer2.to_csv('answer2.csv', index=False)

### Selecionando as colunas com maior correlação

In [65]:
preProcessor2.fit(X_train)
X = pd.DataFrame(preProcessor2.transform(X_train))
X['NU_NOTA_MT'] = y_train.values
correlation = X.corr()[['NU_NOTA_MT']].sort_values(by='NU_NOTA_MT', ascending=False)
# Elevando ao quadrado para observar em valores absolutos quais correlações influenciam mais na nota
correlation = np.power(correlation, 2).sort_values(by='NU_NOTA_MT', ascending=False)
colunas_correlacionadas = correlation.index[1:7]

In [66]:
X_test_final = preProcessor2.transform(X_test)
X_test_final = pd.DataFrame(X_test_final)[colunas_correlacionadas].values

In [67]:
X_preProcessado = X[colunas_correlacionadas].values

In [68]:
regression = RandomForestRegressor()

In [69]:
regression.fit(X_preProcessado, y_train)
notas3 = regression.predict(X_test_final)

In [70]:
answer3 = pd.DataFrame({'NU_INSCRICAO': inscricao, 'NU_NOTA_MT': notas3})
answer3['NU_NOTA_MT'] = answer3['NU_NOTA_MT'].astype('float')

def notaNan(x):
    if x['NU_NOTA_MT'] <= 300:
        return np.nan
    else:
        return round(x['NU_NOTA_MT'], 1)

answer3['NU_NOTA_MT'] = answer3.apply(notaNan, axis=1)
answer3.head()

,NU_INSCRICAO,NU_NOTA_MT
0,73ff9fcc02f0a99919906c942c2e1a1042cdcf98,522.2
1,71a95f9f1b91a82c65ad94abbdf9f54e6066f968,445.0
2,b38a03232f43b11c9d0788abaf060f7366053b6d,575.3
3,70b682d9a3636be23f6120fa9d6b164eb3c6002d,NaN
4,715494628a50142ce8cb17191cfe6d0f3cae0934,692.6


In [71]:
answer3.to_csv('answer3.csv', index=False)

### 2ª Tentativa com o TPOT

In [72]:
X_test_preProcessed = preProcessor2.transform(X_test)

In [73]:
tpot2 = TPOTRegressor(generations=5, max_eval_time_mins= 1, population_size=10, verbosity=2)

In [74]:
tpot2.fit(X_preProcessado, y_train)


Generation 1 - Current best internal CV score: -4486.701882418409
Generation 2 - Current best internal CV score: -4486.701882418409
Generation 3 - Current best internal CV score: -4486.701882418409
Generation 4 - Current best internal CV score: -4474.176231936462
Generation 5 - Current best internal CV score: -4474.176231936462
Best pipeline: GradientBoostingRegressor(input_matrix, alpha=0.8, learning_rate=0.1, loss=ls, max_depth=5, max_features=0.4, min_samples_leaf=10, min_samples_split=8, n_estimators=100, subsample=0.9500000000000001)


TPOTRegressor(config_dict=None, crossover_rate=0.1, cv=5,
              disable_update_check=False, early_stop=None, generations=5,
              log_file=<ipykernel.iostream.OutStream object at 0x0000012A9580D288>,
              max_eval_time_mins=1, max_time_mins=None, memory=None,
              mutation_rate=0.9, n_jobs=1, offspring_size=None,
              periodic_checkpoint_folder=None, population_size=10,
              random_state=None, scoring=None, subsample=1.0, template=None,
              use_dask=False, verbosity=2, warm_start=False)

In [75]:
notas4 = tpot2.predict(X_test_final)
notas4

array([ 4.64413094e+02,  4.48372507e+02,  5.77808884e+02, ...,
        6.73772336e+02,  4.45672898e+02, -8.87303481e-02])

In [76]:
answer4 = pd.DataFrame({'NU_INSCRICAO': inscricao, 'NU_NOTA_MT': notas4})
answer4['NU_NOTA_MT'] = answer4['NU_NOTA_MT'].astype('float')

def notaNan(x):
    if x['NU_NOTA_MT'] <= 300:
        return np.nan
    else:
        return round(x['NU_NOTA_MT'], 1)

answer4['NU_NOTA_MT'] = answer4.apply(notaNan, axis=1)
answer4.head()

,NU_INSCRICAO,NU_NOTA_MT
0,73ff9fcc02f0a99919906c942c2e1a1042cdcf98,464.4
1,71a95f9f1b91a82c65ad94abbdf9f54e6066f968,448.4
2,b38a03232f43b11c9d0788abaf060f7366053b6d,577.8
3,70b682d9a3636be23f6120fa9d6b164eb3c6002d,NaN
4,715494628a50142ce8cb17191cfe6d0f3cae0934,594.4


In [77]:
answer4.to_csv('answer4.csv', index=False)

In [ ]:
# pre projeto da classe pre processamento
class Preprocessamento():
    def __init__(self):
        self.X_treino = None
        self.y_treino = None
        self.X_teste = None
        self.modelo = None
    def preprocessar_treino_sem_imputacao(self, df, y_train):
        df_aux = pd.DataFrame({'Colunas': df.columns, 'Tipos': df.dtypes, 'Na_count': df.isna().sum(), 
                       'Na_percent': df.isna().sum()/df.shape[0]})
        
        # Algumas colunas tem mais de 50% de dados faltantes, imputar os dados não traria beneficio ao modelo de ML, 
        # assim essas colunas serão deletadas
        colComMuitosDadosFaltantes = [*df_aux [df_aux['Na_percent'] >= 0.5]['Colunas'].values]
        print('Deletando {} colunas do dataset de treino e teste, porque mais de 50% de dados estão ausentes'.format(len(colComMuitosDadosFaltantes)))
        print()
        df.drop(colComMuitosDadosFaltantes, axis=1, inplace=True)
        col = [i for i in X_test_df.columns if i in colComMuitosDadosFaltantes]
        X_test_df.drop(col, axis=1, inplace=True)
        print('-*-*-*-*-*-*-*-*-*-*-*-*-*')
        print('Dropando {} linhas em que a nota de matemática do dataset de treino está faltando'.format(df_aux.loc['NU_NOTA_MT']['Na_count']))
        df.dropna(subset=['NU_NOTA_MT'], axis=0, inplace=True)
        
        df = df[[*X_test_df.columns] + ['NU_NOTA_MT']]
        
               
        colunas_dropadas = ['NU_INSCRICAO', 'CO_UF_RESIDENCIA', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT']
        cat = re.compile('SG_|TP_|IN_|Q0')
        colunas_categoricas = [i for i in filter(cat.match, df.columns) if i not in colunas_dropadas]
        num = re.compile('NU_')
        colunas_numericas = [i for i in filter(num.match, df.columns) if i not in colunas_dropadas]
        colunas_dropadas = ['NU_INSCRICAO', 'CO_UF_RESIDENCIA', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT']
        df.drop(colunas_dropadas, axis=1, inplace=True)
        print('O dataset possui {} observações'.format(df.shape[0]))
        df.dropna(inplace=True)
        print('Após excluir as linhas com valores inexistentes, o dataset ainda possui {} observações'.format(df.shape[0]))
        pass
    def preprocessar_teste_sem_imputacao(self, df):
        colunas_dropadas = ['NU_INSCRICAO', 'CO_UF_RESIDENCIA', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT']
        cat = re.compile('SG_|TP_|IN_|Q0')
        colunas_categoricas = [i for i in filter(cat.match, df.columns) if i not in colunas_dropadas]
        num = re.compile('NU_')
        colunas_numericas = [i for i in filter(num.match, df.columns) if i not in colunas_dropadas]
        pass